In [19]:
%run ./init.ipynb

Length of attributes added  100000
Number of valid rows with attributes  97037


In [20]:
missingIDs = np.where(pd.isnull(df['Background']))[0]
print(missingIDs)

[14345 44349 59365 ... 99997 99998 99999]


In [21]:
listing = {}

In [22]:
%%time
startRange = missingIDs[0]
stopRange = 100001

refStartx = startRange

#for x in range(startRange, stopRange):
for x in (missingIDs):
    scraper = cloudscraper.create_scraper(
        delay=1,
        browser={
            'browser': 'firefox',
            'platform': 'windows',
            'mobile': False
        })
    html = scraper.get("https://niftys.com/nft/0x39ceaa47306381b6d79ad46af0f36bc5332386f2/" + str(x + 1)).text

    soup = BeautifulSoup(html, 'lxml')

    rows = soup.find_all('div', 'label')

    rowAttributes = {}
    for row in rows:
        cells = row
        str_row = str(row)
        value = str_row.split('!-- -->')
        cleanTextAttribute = BeautifulSoup(value[0], "lxml").get_text()
        if cleanTextAttribute == 'Available on PALM':
            continue
        if (len(value) > 1):
            cleanTextValue = BeautifulSoup(value[1], "lxml").get_text().strip()
        else:
            cleanTextValue = ""
        
        rowAttributes[cleanTextAttribute] = cleanTextValue
    
    output_list = [(rowAttributes[key]) for key in (manualColumns) if key in rowAttributes]

    
    listing[str(x+1)] = output_list
    clear_output(wait=True)
    print(x+1)
print(listing)
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColumns)  #pd.DataFrame(listing, orient='index')
data_dictionary.to_csv('MissingAttributes_' + str(missingIDs[0]) + '_' + str(missingIDs[len(missingIDs)-1]) + '.csv')
refStartx = x+1
listing.clear()

97051


KeyboardInterrupt: 

In [23]:
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColumns)  #pd.DataFrame(listing, orient='index')
data_dictionary.to_csv('MissingAttributes_' + str(missingIDs[0]) + '_' + str(missingIDs[len(missingIDs)-1]) + '.csv')

In [6]:
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index')  #pd.DataFrame(listing, orient='index')

In [7]:
for count in range(0, len(manualColumns)):
    print(count)
    print(data_dictionary[manualColumns[count]].value_counts())
    data_dictionary[manualColumns[count]].value_counts(normalize=True)
    print('---------------------------------')

0


KeyError: 'Background'

In [ ]:
print('done')